In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import *
from tensorflow.keras.layers import LSTM
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.regularizers import l2

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [8]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [9]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [10]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [11]:
run_experiment(X,y)

Epoch 1/50
8/8 [==============================] - 1s 64ms/step - loss: 10.3305 - accuracy: 0.5082 - precision: 0.4697 - recall: 0.3808 - auc: 0.5082
Epoch 2/50
8/8 [==============================] - 1s 68ms/step - loss: 6.0461 - accuracy: 0.5235 - precision: 0.4928 - recall: 0.5631 - auc: 0.5487
Epoch 3/50
8/8 [==============================] - 1s 63ms/step - loss: 4.4186 - accuracy: 0.6145 - precision: 0.5760 - recall: 0.6729 - auc: 0.6644
Epoch 4/50
8/8 [==============================] - 0s 61ms/step - loss: 3.7836 - accuracy: 0.6725 - precision: 0.6210 - recall: 0.7734 - auc: 0.7352
Epoch 5/50
8/8 [==============================] - 1s 64ms/step - loss: 3.2635 - accuracy: 0.7809 - precision: 0.7579 - recall: 0.7827 - auc: 0.8637
Epoch 6/50
8/8 [==============================] - 0s 59ms/step - loss: 2.8560 - accuracy: 0.8521 - precision: 0.8399 - recall: 0.8458 - auc: 0.9155
Epoch 7/50
8/8 [==============================] - 1s 68ms/step - loss: 2.5407 - accuracy: 0.9189 - precision: 0

8/8 [==============================] - 1s 73ms/step - loss: 3.2426 - accuracy: 0.7678 - precision_1: 0.7160 - recall_1: 0.8364 - auc_1: 0.8533
Epoch 6/50
8/8 [==============================] - 1s 66ms/step - loss: 2.8099 - accuracy: 0.8697 - precision_1: 0.8670 - recall_1: 0.8528 - auc_1: 0.9399
Epoch 7/50
8/8 [==============================] - 0s 56ms/step - loss: 2.4980 - accuracy: 0.9168 - precision_1: 0.9000 - recall_1: 0.9252 - auc_1: 0.9673
Epoch 8/50
8/8 [==============================] - 1s 68ms/step - loss: 2.3591 - accuracy: 0.9047 - precision_1: 0.8974 - recall_1: 0.8995 - auc_1: 0.9630
Epoch 9/50
8/8 [==============================] - 1s 70ms/step - loss: 2.3023 - accuracy: 0.8620 - precision_1: 0.8813 - recall_1: 0.8154 - auc_1: 0.9460
Epoch 10/50
8/8 [==============================] - 1s 73ms/step - loss: 2.1307 - accuracy: 0.9255 - precision_1: 0.9167 - recall_1: 0.9252 - auc_1: 0.9728
Epoch 11/50
8/8 [==============================] - 0s 57ms/step - loss: 2.0016 - accur

Epoch 8/50
8/8 [==============================] - 1s 67ms/step - loss: 2.4136 - accuracy: 0.9430 - precision_2: 0.9372 - recall_2: 0.9416 - auc_2: 0.9783
Epoch 9/50
8/8 [==============================] - 1s 74ms/step - loss: 2.2755 - accuracy: 0.9387 - precision_2: 0.9266 - recall_2: 0.9439 - auc_2: 0.9799
Epoch 10/50
8/8 [==============================] - 1s 70ms/step - loss: 2.1604 - accuracy: 0.9365 - precision_2: 0.9243 - recall_2: 0.9416 - auc_2: 0.9851
Epoch 11/50
8/8 [==============================] - 1s 69ms/step - loss: 2.0311 - accuracy: 0.9606 - precision_2: 0.9645 - recall_2: 0.9509 - auc_2: 0.9897
Epoch 12/50
8/8 [==============================] - 1s 69ms/step - loss: 1.9245 - accuracy: 0.9617 - precision_2: 0.9602 - recall_2: 0.9579 - auc_2: 0.9905
Epoch 13/50
8/8 [==============================] - 1s 66ms/step - loss: 1.9295 - accuracy: 0.9288 - precision_2: 0.9481 - recall_2: 0.8972 - auc_2: 0.9742
Epoch 14/50
8/8 [==============================] - 1s 78ms/step - loss: 

8/8 [==============================] - 0s 51ms/step - loss: 2.1574 - accuracy: 0.9452 - precision_3: 0.9543 - recall_3: 0.9276 - auc_3: 0.9847
Epoch 11/50
8/8 [==============================] - 0s 60ms/step - loss: 2.0412 - accuracy: 0.9485 - precision_3: 0.9568 - recall_3: 0.9322 - auc_3: 0.9835
Epoch 12/50
8/8 [==============================] - 1s 63ms/step - loss: 1.9066 - accuracy: 0.9639 - precision_3: 0.9540 - recall_3: 0.9696 - auc_3: 0.9902
Epoch 13/50
8/8 [==============================] - 1s 63ms/step - loss: 1.8132 - accuracy: 0.9682 - precision_3: 0.9650 - recall_3: 0.9673 - auc_3: 0.9917
Epoch 14/50
8/8 [==============================] - 1s 71ms/step - loss: 1.7533 - accuracy: 0.9595 - precision_3: 0.9515 - recall_3: 0.9626 - auc_3: 0.9898
Epoch 15/50
8/8 [==============================] - 1s 68ms/step - loss: 1.7113 - accuracy: 0.9573 - precision_3: 0.9534 - recall_3: 0.9556 - auc_3: 0.9882
Epoch 16/50
8/8 [==============================] - 1s 63ms/step - loss: 1.6329 - a

Epoch 13/50
8/8 [==============================] - 0s 56ms/step - loss: 1.8050 - accuracy: 0.9266 - precision_4: 0.9093 - recall_4: 0.9369 - auc_4: 0.9755
Epoch 14/50
8/8 [==============================] - 1s 82ms/step - loss: 1.6823 - accuracy: 0.9704 - precision_4: 0.9740 - recall_4: 0.9626 - auc_4: 0.9918
Epoch 15/50
8/8 [==============================] - 1s 64ms/step - loss: 1.5920 - accuracy: 0.9671 - precision_4: 0.9671 - recall_4: 0.9626 - auc_4: 0.9935
Epoch 16/50
8/8 [==============================] - 0s 62ms/step - loss: 1.5092 - accuracy: 0.9660 - precision_4: 0.9671 - recall_4: 0.9603 - auc_4: 0.9939
Epoch 17/50
8/8 [==============================] - 1s 82ms/step - loss: 1.5023 - accuracy: 0.9507 - precision_4: 0.9323 - recall_4: 0.9650 - auc_4: 0.9866
Epoch 18/50
8/8 [==============================] - 1s 86ms/step - loss: 1.4568 - accuracy: 0.9573 - precision_4: 0.9642 - recall_4: 0.9439 - auc_4: 0.9883
Epoch 19/50
8/8 [==============================] - 1s 79ms/step - loss

In [12]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),input_shape=(None,n_length,n_features))))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(100, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)
    
run_experiment_2(X,y)

Epoch 1/50
8/8 [==============================] - 1s 86ms/step - loss: 10.5451 - accuracy: 0.5104 - precision_5: 0.4545 - recall_5: 0.2220 - auc_5: 0.5323
Epoch 2/50
8/8 [==============================] - 1s 69ms/step - loss: 6.4513 - accuracy: 0.5181 - precision_5: 0.4860 - recall_5: 0.4860 - auc_5: 0.5282
Epoch 3/50
8/8 [==============================] - 1s 74ms/step - loss: 4.6756 - accuracy: 0.5915 - precision_5: 0.5514 - recall_5: 0.6893 - auc_5: 0.6494
Epoch 4/50
8/8 [==============================] - 1s 82ms/step - loss: 3.8945 - accuracy: 0.6594 - precision_5: 0.6163 - recall_5: 0.7243 - auc_5: 0.7425
Epoch 5/50
8/8 [==============================] - 1s 73ms/step - loss: 3.3455 - accuracy: 0.7525 - precision_5: 0.7149 - recall_5: 0.7850 - auc_5: 0.8335
Epoch 6/50
8/8 [==============================] - 1s 66ms/step - loss: 2.9198 - accuracy: 0.8445 - precision_5: 0.7907 - recall_5: 0.9089 - auc_5: 0.9265
Epoch 7/50
8/8 [==============================] - 1s 63ms/step - loss: 2.61

Epoch 1/50
8/8 [==============================] - 0s 61ms/step - loss: 10.4904 - accuracy: 0.4962 - precision_6: 0.4563 - recall_6: 0.3902 - auc_6: 0.5088
Epoch 2/50
8/8 [==============================] - 1s 69ms/step - loss: 6.3434 - accuracy: 0.6221 - precision_6: 0.6089 - recall_6: 0.5421 - auc_6: 0.6698
Epoch 3/50
8/8 [==============================] - 1s 72ms/step - loss: 4.6465 - accuracy: 0.7393 - precision_6: 0.7093 - recall_6: 0.7523 - auc_6: 0.8087
Epoch 4/50
8/8 [==============================] - 1s 93ms/step - loss: 3.9443 - accuracy: 0.7963 - precision_6: 0.8010 - recall_6: 0.7523 - auc_6: 0.8799
Epoch 5/50
8/8 [==============================] - 1s 84ms/step - loss: 3.4567 - accuracy: 0.8784 - precision_6: 0.8856 - recall_6: 0.8505 - auc_6: 0.9429
Epoch 6/50
8/8 [==============================] - 1s 70ms/step - loss: 3.0694 - accuracy: 0.9211 - precision_6: 0.9140 - recall_6: 0.9182 - auc_6: 0.9630
Epoch 7/50
8/8 [==============================] - 1s 68ms/step - loss: 2.77

Epoch 1/50
8/8 [==============================] - 1s 75ms/step - loss: 10.6514 - accuracy: 0.5257 - precision_7: 0.4942 - recall_7: 0.4953 - auc_7: 0.5123
Epoch 2/50
8/8 [==============================] - 1s 70ms/step - loss: 6.4856 - accuracy: 0.5849 - precision_7: 0.5602 - recall_7: 0.5327 - auc_7: 0.6240
Epoch 3/50
8/8 [==============================] - 1s 64ms/step - loss: 4.6410 - accuracy: 0.6955 - precision_7: 0.6697 - recall_7: 0.6916 - auc_7: 0.7573
Epoch 4/50
8/8 [==============================] - 1s 67ms/step - loss: 3.9005 - accuracy: 0.7163 - precision_7: 0.6952 - recall_7: 0.7033 - auc_7: 0.7993
Epoch 5/50
8/8 [==============================] - 1s 73ms/step - loss: 3.3336 - accuracy: 0.8620 - precision_7: 0.8386 - recall_7: 0.8738 - auc_7: 0.9319
Epoch 6/50
8/8 [==============================] - 1s 63ms/step - loss: 2.9473 - accuracy: 0.9014 - precision_7: 0.8912 - recall_7: 0.8995 - auc_7: 0.9487
Epoch 7/50
8/8 [==============================] - 1s 76ms/step - loss: 2.70

Epoch 1/50
8/8 [==============================] - 1s 71ms/step - loss: 10.4148 - accuracy: 0.4918 - precision_8: 0.4552 - recall_8: 0.4276 - auc_8: 0.5109
Epoch 2/50
8/8 [==============================] - 1s 74ms/step - loss: 6.1551 - accuracy: 0.6068 - precision_8: 0.5816 - recall_8: 0.5748 - auc_8: 0.6432
Epoch 3/50
8/8 [==============================] - 1s 78ms/step - loss: 4.4371 - accuracy: 0.6955 - precision_8: 0.6689 - recall_8: 0.6939 - auc_8: 0.7754
Epoch 4/50
8/8 [==============================] - 1s 75ms/step - loss: 3.7007 - accuracy: 0.8171 - precision_8: 0.7680 - recall_8: 0.8738 - auc_8: 0.9079
Epoch 5/50
8/8 [==============================] - 1s 81ms/step - loss: 3.2585 - accuracy: 0.8576 - precision_8: 0.8565 - recall_8: 0.8364 - auc_8: 0.9363
Epoch 6/50
8/8 [==============================] - 1s 66ms/step - loss: 2.8513 - accuracy: 0.9288 - precision_8: 0.9192 - recall_8: 0.9299 - auc_8: 0.9780
Epoch 7/50
8/8 [==============================] - 1s 77ms/step - loss: 2.60

Epoch 1/50
8/8 [==============================] - 1s 73ms/step - loss: 10.4059 - accuracy: 0.5279 - precision_9: 0.4964 - recall_9: 0.4790 - auc_9: 0.5259
Epoch 2/50
8/8 [==============================] - 0s 59ms/step - loss: 6.1165 - accuracy: 0.5652 - precision_9: 0.5522 - recall_9: 0.3832 - auc_9: 0.5891
Epoch 3/50
8/8 [==============================] - 1s 64ms/step - loss: 4.4514 - accuracy: 0.6221 - precision_9: 0.5933 - recall_9: 0.6168 - auc_9: 0.6696
Epoch 4/50
8/8 [==============================] - 1s 77ms/step - loss: 3.7880 - accuracy: 0.6966 - precision_9: 0.6837 - recall_9: 0.6565 - auc_9: 0.7600
Epoch 5/50
8/8 [==============================] - 1s 66ms/step - loss: 3.3403 - accuracy: 0.7426 - precision_9: 0.6817 - recall_9: 0.8458 - auc_9: 0.8064
Epoch 6/50
8/8 [==============================] - 1s 70ms/step - loss: 2.8890 - accuracy: 0.8642 - precision_9: 0.8333 - recall_9: 0.8879 - auc_9: 0.9416
Epoch 7/50
8/8 [==============================] - 0s 53ms/step - loss: 2.57

 Accuracy: 94.760% (+/-5.943) 
 Precision: 90.354% (+/-10.636) 
 Recall: 98.864% (+/-1.245) 
 AUC: 99.728% (+/-0.212) 


In [13]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='selu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_3(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)
    
run_experiment_3(X,y)

Epoch 1/50
8/8 [==============================] - 1s 67ms/step - loss: 11.5042 - accuracy: 0.4962 - precision_10: 0.4608 - recall_10: 0.4393 - auc_10: 0.5040
Epoch 2/50
8/8 [==============================] - 1s 67ms/step - loss: 8.0935 - accuracy: 0.6145 - precision_10: 0.5892 - recall_10: 0.5864 - auc_10: 0.6586
Epoch 3/50
8/8 [==============================] - 1s 68ms/step - loss: 5.8668 - accuracy: 0.7240 - precision_10: 0.6888 - recall_10: 0.7500 - auc_10: 0.8100
Epoch 4/50
8/8 [==============================] - 1s 74ms/step - loss: 4.6901 - accuracy: 0.7886 - precision_10: 0.7617 - recall_10: 0.7991 - auc_10: 0.8841
Epoch 5/50
8/8 [==============================] - 0s 62ms/step - loss: 3.9700 - accuracy: 0.8554 - precision_10: 0.8364 - recall_10: 0.8598 - auc_10: 0.9429
Epoch 6/50
8/8 [==============================] - 1s 73ms/step - loss: 3.4948 - accuracy: 0.9343 - precision_10: 0.9299 - recall_10: 0.9299 - auc_10: 0.9798
Epoch 7/50
8/8 [==============================] - 1s 73ms

Epoch 1/50
8/8 [==============================] - 0s 54ms/step - loss: 11.6311 - accuracy: 0.5148 - precision_11: 0.4835 - recall_11: 0.5140 - auc_11: 0.5184
Epoch 2/50
8/8 [==============================] - 1s 63ms/step - loss: 8.3996 - accuracy: 0.6320 - precision_11: 0.5954 - recall_11: 0.6706 - auc_11: 0.6704
Epoch 3/50
8/8 [==============================] - 1s 67ms/step - loss: 6.1880 - accuracy: 0.7130 - precision_11: 0.6895 - recall_11: 0.7056 - auc_11: 0.7753
Epoch 4/50
8/8 [==============================] - 1s 73ms/step - loss: 4.8666 - accuracy: 0.7996 - precision_11: 0.7692 - recall_11: 0.8178 - auc_11: 0.8859
Epoch 5/50
8/8 [==============================] - 1s 63ms/step - loss: 4.0669 - accuracy: 0.8795 - precision_11: 0.8614 - recall_11: 0.8855 - auc_11: 0.9492
Epoch 6/50
8/8 [==============================] - 1s 66ms/step - loss: 3.5527 - accuracy: 0.9244 - precision_11: 0.9165 - recall_11: 0.9229 - auc_11: 0.9803
Epoch 7/50
8/8 [==============================] - 1s 71ms

Epoch 1/50
8/8 [==============================] - 0s 61ms/step - loss: 11.2679 - accuracy: 0.6057 - precision_12: 0.5810 - recall_12: 0.5701 - auc_12: 0.6378
Epoch 2/50
8/8 [==============================] - 0s 56ms/step - loss: 7.8775 - accuracy: 0.7448 - precision_12: 0.6978 - recall_12: 0.8037 - auc_12: 0.8301
Epoch 3/50
8/8 [==============================] - 0s 57ms/step - loss: 5.7385 - accuracy: 0.8346 - precision_12: 0.8031 - recall_12: 0.8575 - auc_12: 0.9193
Epoch 4/50
8/8 [==============================] - 0s 51ms/step - loss: 4.5871 - accuracy: 0.9135 - precision_12: 0.9087 - recall_12: 0.9065 - auc_12: 0.9756
Epoch 5/50
8/8 [==============================] - 0s 53ms/step - loss: 4.0119 - accuracy: 0.9157 - precision_12: 0.8874 - recall_12: 0.9393 - auc_12: 0.9759
Epoch 6/50
8/8 [==============================] - 0s 58ms/step - loss: 3.5296 - accuracy: 0.9748 - precision_12: 0.9833 - recall_12: 0.9626 - auc_12: 0.9955
Epoch 7/50
8/8 [==============================] - 0s 57ms

Epoch 1/50
8/8 [==============================] - 1s 71ms/step - loss: 11.6818 - accuracy: 0.5170 - precision_13: 0.4870 - recall_13: 0.5701 - auc_13: 0.5143
Epoch 2/50
8/8 [==============================] - 1s 64ms/step - loss: 8.4543 - accuracy: 0.5663 - precision_13: 0.5465 - recall_13: 0.4393 - auc_13: 0.6013
Epoch 3/50
8/8 [==============================] - 0s 59ms/step - loss: 6.2485 - accuracy: 0.6265 - precision_13: 0.5841 - recall_13: 0.7056 - auc_13: 0.6947
Epoch 4/50
8/8 [==============================] - 1s 67ms/step - loss: 4.9206 - accuracy: 0.7547 - precision_13: 0.7208 - recall_13: 0.7780 - auc_13: 0.8391
Epoch 5/50
8/8 [==============================] - 1s 66ms/step - loss: 4.1159 - accuracy: 0.8620 - precision_13: 0.8479 - recall_13: 0.8598 - auc_13: 0.9327
Epoch 6/50
8/8 [==============================] - 0s 58ms/step - loss: 3.6000 - accuracy: 0.9343 - precision_13: 0.9259 - recall_13: 0.9346 - auc_13: 0.9770
Epoch 7/50
8/8 [==============================] - 0s 62ms

Epoch 1/50
8/8 [==============================] - 0s 55ms/step - loss: 11.3231 - accuracy: 0.5509 - precision_14: 0.5215 - recall_14: 0.5093 - auc_14: 0.5846
Epoch 2/50
8/8 [==============================] - 0s 55ms/step - loss: 7.9155 - accuracy: 0.7152 - precision_14: 0.7165 - recall_14: 0.6495 - auc_14: 0.8154
Epoch 3/50
8/8 [==============================] - 0s 53ms/step - loss: 5.7466 - accuracy: 0.8116 - precision_14: 0.7807 - recall_14: 0.8318 - auc_14: 0.8943
Epoch 4/50
8/8 [==============================] - 0s 54ms/step - loss: 4.5104 - accuracy: 0.9091 - precision_14: 0.8842 - recall_14: 0.9276 - auc_14: 0.9695
Epoch 5/50
8/8 [==============================] - 0s 58ms/step - loss: 3.8312 - accuracy: 0.9409 - precision_14: 0.9349 - recall_14: 0.9393 - auc_14: 0.9884
Epoch 6/50
8/8 [==============================] - 0s 52ms/step - loss: 3.4097 - accuracy: 0.9606 - precision_14: 0.9537 - recall_14: 0.9626 - auc_14: 0.9944
Epoch 7/50
8/8 [==============================] - 0s 57ms

 Accuracy: 58.428% (+/-3.398) 
 Precision: 48.128% (+/-2.106) 
 Recall: 99.773% (+/-0.455) 
 AUC: 89.443% (+/-1.573) 
